<a href="https://colab.research.google.com/github/jeffblackadar/CRANE-CCAD-maps/blob/main/evernote_extract_images_with_keys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evernote: Extract Images from a Notebook

## Assumptions:
* The Evernote notebook is a regular notebook and in production.
* You have production keys that are also _activated_ for production.  ("We have activated your API key (username-xxxx) on our production service.")

## Function
* This program lists all of the notebooks.
* For a selected notebook, it loops through all notes in it.
* For each note, it saves images known as "resources" as a file.

## Credit to:
### Evernote API & Python 3.x, https://xbopp.com/evernote-to-python1/
### Evernote Developer Documentation https://dev.evernote.com/doc/
### Evernote API reference https://dev.evernote.com/doc/reference/
### Search https://dev.evernote.com/doc/articles/search.php
### Resources https://dev.evernote.com/doc/articles/resources.php
### Developer token https://sandbox.evernote.com/api/DeveloperToken.action
### Get keys https://dev.evernote.com/doc/articles/authentication.php

In [1]:
!pip install git+https://github.com/evernote/evernote-sdk-python3.git
!pip install oauth2

  Cloning https://github.com/evernote/evernote-sdk-python3.git to /tmp/pip-req-build-1_u3d7ea
  Running command git clone -q https://github.com/evernote/evernote-sdk-python3.git /tmp/pip-req-build-1_u3d7ea
  Created wheel for evernote3: filename=evernote3-1.25.0-cp36-none-any.whl size=143704 sha256=3a613f47dedcd144bffafcda54247ff458aa09fd52c535d0bfcbbff6c0fe018b
  Stored in directory: /tmp/pip-ephem-wheel-cache-6s72jjbz/wheels/63/4a/27/69fc0e9c3afe2951dcfc496e595173fe71a3d456f31c8d0666
Successfully built evernote3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Store keys
* Make a directory for keys called "evernote_export"
* Store key file in /content/drive/MyDrive/evernote_export/evernote_api.json



In [4]:
import json
import evernote.edam.userstore.constants as UserStoreConstants
import os
from urllib import parse
from evernote.api.client import EvernoteClient
from evernote.edam.notestore.ttypes import NoteFilter, NotesMetadataResultSpec
import evernote.edam.userstore.UserStore as UserStore
import evernote.edam.notestore.NoteStore as NoteStore
import evernote.edam.userstore.constants as UserStoreConstants
import thrift.protocol.TBinaryProtocol as TBinaryProtocol

# settings_path holds where the keys are.  This will be the folder where images are saved.
settings_path = "/content/drive/MyDrive/evernote_export/"
# This is the exact name of the notebook to export
target_notebook_title ="jeffblackadar's notebook"

evernote_settings_file = settings_path+'evernote_api.json'
with open(evernote_settings_file, "r") as read_file:
    evernote_auth_data = json.load(read_file)

consumer_key_json = evernote_auth_data["consumer_key"]
consumer_secret_json = evernote_auth_data["consumer_secret"]
application_name_json = evernote_auth_data["application_name"]

print("Check you have the right application name: ",application_name_json)

#************************
# sandbox = True, for production
# sandbox = False, for development
#************************

client = EvernoteClient(
    consumer_key=consumer_key_json,
    consumer_secret=consumer_secret_json,
    sandbox = False,
    china = False   

)

request_token = client.get_request_token(callback_url='https://jeffblackadar.ca')

auth_url = client.get_authorize_url(request_token)
print("*******************************************************")
print("* click on the URL below.  Then copy the whole URL. Paste it into   url =""   (This is a manual way to make Oauth happen without writing a full client application.)")
print("*   |")
print("*  \|/")
print("*   V")
print(auth_url)

Check you have the right application name:  extract_images
*******************************************************
* click on the URL below.  Then copy the whole URL. Paste it into   url =   (This is a manual way to make Oauth happen without writing a full client application.)
*   |
*  \|/
*   V
https://www.evernote.com/OAuth.action?oauth_token=jeffblackadar-5809.1776F696C3C.68747470733A2F2F6A656666626C61636B616461722E6361.592934653ABA1DBD1F37F310B7964FCB


In [5]:
# Paste the URL in here
#          |
#         \|/
#          V
url = "https://jeffblackadar.ca/?oauth_token=jeffblackadar-5809.1776F696C3C.68747470733A2F2F6A656666626C61636B616461722E6361.592934653ABA1DBD1F37F310B7964FCB&oauth_verifier=3C5E28FCF40F063FE4938F1480456D9C&sandbox_lnb=false"

request_dict = dict(parse.parse_qs(parse.urlsplit(url).query))

oauth_verifier =request_dict.get('oauth_verifier','')[0]
access_token = client.get_access_token(
    request_token['oauth_token'],
    request_token['oauth_token_secret'],
    request_dict.get('oauth_verifier','')[0]
)


In [6]:
import io
from PIL import Image
client = EvernoteClient(token=access_token, sandbox=False, china=False, service_host="www.evernote.com")
#client = EvernoteClient(token=access_token)

user_store = client.get_user_store()
version_ok = user_store.checkVersion(
    "Evernote EDAMTest (Python)",
    UserStoreConstants.EDAM_VERSION_MAJOR,
    UserStoreConstants.EDAM_VERSION_MINOR
)
print("Is my Evernote API version up to date? ", str(version_ok))
if not version_ok:
    exit(1)

note_store = client.get_note_store()

# List all of the notebooks in the user's account
notebooks = note_store.listNotebooks()
print("Found ", len(notebooks), " notebooks:")
myNotebookGuid = ""
for notebook in notebooks:
    print("notebook title: ",notebook.name, "guid: ", notebook.guid)
    # look for a specific notebook
    if(notebook.name == target_notebook_title):
        myNotebookGuid = notebook.guid    

if myNotebookGuid == "":
    print("Specific notebook note found. Is the name spelled right?")
    exit(1)

#sort of pointless - since we already have a handle on it above.
myNotebook = note_store.getNotebook(myNotebookGuid)

# filter to get notes inside the notebook
filter = NoteFilter()
filter.notebookGuid = myNotebookGuid
filter.ascending = False

spec = NotesMetadataResultSpec()
spec.includeTitle = True

# May need to change
ourNoteList = note_store.findNotesMetadata(access_token, filter, 0, 100, spec)

image_destination = settings_path

for note in ourNoteList.notes:
    wholeNote = note_store.getNote(access_token, note.guid, True, True, True, True) 
    print ("Content length: %d" % wholeNote.contentLength)
    note_resources = wholeNote.resources
    for note_resource in note_resources:
        file_content = note_resource.data.body
        file_type = note_resource.mime
        file_name = note_resource.attributes.fileName
        if(file_name is None):
            print("Note resource with file_name = None")  
        else:
            # Thanks to https://stackoverflow.com/questions/42339876/error-unicodedecodeerror-utf-8-codec-cant-decode-byte-0xff-in-position-0-in
            decode_bytes = file_content.decode('ISO-8859-1')
            resource_bytes = bytes(decode_bytes, 'ISO-8859-1')
            stream = io.BytesIO(resource_bytes)
            img = Image.open(stream)
            print("saving",file_name)
            img.save(os.path.join(image_destination,file_name))

Is my Evernote API version up to date?  True
Found  1  notebooks:
notebook title:  jeffblackadar's notebook guid:  28bce459-9f54-448a-b19d-417a22e99b14
Content length: 260
saving BlueMtHearthforShawnJeff.jpg
Content length: 10451
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
Note resource with file_name = None
